In [52]:
import json
import sklearn
import numpy
import random
import scipy
import sklearn.datasets
import sklearn.feature_extraction.text
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.metrics
import sklearn.utils
import string

import matplotlib.pyplot as plt

In [53]:
with open('articles_with_texts_final.json') as data_file:
    articles = json.load(data_file)

In [54]:
article_keys = articles[u'data'].keys()
vectorizer = \
sklearn.feature_extraction.text.TfidfVectorizer(sublinear_tf=True, 
                                                max_df=0.5, 
                                                stop_words='english')
_ = vectorizer.fit(articles[u'data'][k][u'text'] for k in article_keys)

In [55]:
T_keys = [k for k in article_keys if (articles[u'data'][k]['most_emailed'] or articles[u'data'][k]['most_shared'] or \
                articles[u'data'][k]['most_viewed'])]
F_keys = [k for k in article_keys if not (articles[u'data'][k]['most_emailed'] or articles[u'data'][k]['most_shared'] or \
                articles[u'data'][k]['most_viewed'])]

In [56]:
#def feature1(c_articles) : #c_articles is [articles[u'data'][k] for k in keys]

#author_vectorizer = sklearn.feature_extraction.text.CountVectorizer()
#_ = author_vectorizer.fit(string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in [articles[u'data'][k] for k in article_keys])

def author(cas):
    return author_vectorizer.transform(string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in cas)
    
features = []



def getX(key_list):
    x = vectorizer.transform(articles[u'data'][k][u'text'] for k in key_list)
    c_articles = [articles[u'data'][k] for k in key_list]
    for f in features :
        x_new = f(c_articles)
        x = scipy.sparse.hstack([x, x_new])
    return x

In [57]:
#[string.replace(string.replace(ca[u'byline'][u'original'], ' ', ''), '.', '') if ca[u'byline'] != [] else "" for ca in [articles[u'data'][k] for k in article_keys[:4000]]]

[u'ByRUKMINICALLIMACHI',
 '',
 '',
 u'ByERIKPIEPENBURG',
 u'ByPUI-WINGTAM',
 u'ByANGELICAJADEBASTI\xc9N',
 u'ByJULIAWALLACE',
 u'ByNICHOLASFANDOS',
 u'ByKATHLEENROONEY',
 u'ByGEORGEHAY',
 u'ByJENNIFERSEY',
 u'ByPATRICKMcGEEHAN',
 u'ByAMIETSANG',
 '',
 u'ByJANEGORDONJULIEN',
 u'ByERNESTOLONDO\xd1O',
 u'ByEMILYBRENNAN',
 u'ByTHENEWYORKTIMES',
 u'ByCLIFFORDKRAUSSandDIANECARDWELL',
 u'ByZACHWICHTER',
 u'ByJACKANDERSON',
 u'ByPATRICKCHAPPATTE',
 '',
 u'ByCARLOSTEJADA',
 u'ByMAGGIEHABERMAN',
 u'ByMATTFLEGENHEIMERandALANRAPPEPORT',
 u'ByPETERBAKER',
 u'ByTHEASSOCIATEDPRESS',
 u'ByELIZABETHAHARRIS',
 '',
 u'BySTEPHYIN',
 u'ByGREGORAISCHandANJALISINGHVI',
 u'ByJOHNWILLIAMS',
 u'ByANNEBARNARD',
 u'BySHIVANIVORA',
 u'ByKATHERINEROSMAN',
 u'ByBEESHAPIRO',
 u'ByDEBAMLEN',
 u'ByHERALDOMU\xd1OZ',
 u'ByBENHUBBARDandSHUAIBALMOSAWA',
 '',
 '',
 u'ByWILLIAMNEUMANandJDAVIDGOODMAN',
 u'ByINSIDERSTAFF',
 u'ByWILLIAMKRASHBAUM',
 u'ByROBERTCYRAN',
 u'ByELISABETHVINCENTELLI',
 u'ByJESSEMcKINLEYandLISAWFODERARO

In [58]:
X_train = getX(article_keys[:4000])
y_train = [k in T_keys for k in article_keys[:4000]]
X_dev = getX(article_keys[4000:])
y_dev = [k in T_keys for k in article_keys[4000:]]

In [59]:
classifier = \
sklearn.linear_model.SGDClassifier(loss="log",
                                   penalty="elasticnet",
                                   n_iter=5)

_ = classifier.fit(X_train, y_train)

In [60]:
pred = classifier.predict(X_dev)
print sklearn.metrics.accuracy_score(y_dev, pred)

0.754459601259


In [64]:
index_to_word = vectorizer.get_feature_names()
best_elts = numpy.argpartition(classifier.coef_, classifier.coef_.size - 20)[0][-20:]
best_words = [index_to_word[x] for x in best_elts]
best_words

[u'executives',
 u'victims',
 u'interview',
 u'ran',
 u'twitter',
 u'obama',
 u'obamacare',
 u'statement',
 u'far',
 u'millions',
 u'reported',
 u'netflix',
 u'replace',
 u'dr',
 u'designed',
 u'asked',
 u'white',
 u'trump',
 u'told',
 u'poor']

In [65]:
worst_elts = numpy.argpartition(classifier.coef_, 19)[0][:20]
worst_words = [index_to_word[x] for x in worst_elts]
worst_words

[u'season',
 u'_____',
 u'editor',
 u'china',
 u'article',
 u'play',
 u'april',
 u'writer',
 u'march',
 u'york',
 u'european',
 u'prison',
 u'blasio',
 u'puzzle',
 u'film',
 u'com',
 u'trial',
 u'bank',
 u'city',
 u'nytimes']

In [62]:
author([articles[u'data'][k] for k in T_keys])

<1687x1652 sparse matrix of type '<type 'numpy.int64'>'
	with 1783 stored elements in Compressed Sparse Row format>

In [63]:
author([articles[u'data'][k] for k in F_keys])

<3266x1652 sparse matrix of type '<type 'numpy.int64'>'
	with 2859 stored elements in Compressed Sparse Row format>